In [0]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.autograd import Variable
torch.cuda.is_available()

True

In [0]:
train_dataset= datasets.MNIST(root='./data', train=True, transform=transforms.ToTensor(), download=True)
test_dataset= datasets.MNIST(root='./data', train=False, transform=transforms.ToTensor())

0it [00:00, ?it/s]

9920512it [00:01, 8792432.81it/s]                            


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz


  0%|          | 0/28881 [00:00<?, ?it/s]

32768it [00:00, 135716.04it/s]           
  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz


1654784it [00:00, 2189782.63it/s]                           
0it [00:00, ?it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz


8192it [00:00, 50679.80it/s]            


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz
Processing...
Done!


In [0]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once in a notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once in a notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Create & upload a text file.
uploaded = drive.CreateFile({'title': 'Sample file.txt'})
uploaded.SetContentString('Sample upload file content')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

In [0]:
batch_size=100
epochs=10
train_load=torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_load=torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [0]:
print("Number of images in training set: {}".format(len(train_dataset)))
print("Number of images in test set: {}".format(len(test_dataset)))
print("Number of batches in the train loader: {}".format(len(train_load)))
print("Number of batches in the test loader: {}".format(len(test_load)))

Number of images in training set: 60000
Number of images in test set: 10000
Number of batches in the train loader: 600
Number of batches in the test loader: 100


In [0]:
class CNN(nn.Module):
  def __init__(self):
    super(CNN,self).__init__()
    # input_size:28, same_padding=(filter_size-1)/2, 3-1/2=1:padding
    self.cnn1=nn.Conv2d(in_channels=1, out_channels=8, kernel_size=3, stride=1, padding=1)
    # input_size-filter_size +2(padding)/stride + 1 = 28-3+2(1)/1+1=28
    self.batchnorm1=nn.BatchNorm2d(8)
    # output_channel:8, batch(8)
    self.relu=nn.ReLU()
    self.maxpool1=nn.MaxPool2d(kernel_size=2)
    #input_size=28/2=14
    self.cnn2=nn.Conv2d(in_channels=8, out_channels=32, kernel_size=5, stride=1, padding=2)
    # same_padding: (5-1)/2=2:padding_size. 
    self.batchnorm2=nn.BatchNorm2d(32)
    self.maxpool2=nn.MaxPool2d(kernel_size=2)
    # input_size=14/2=7
    # 32x7x7=1568
    self.fc1 =nn.Linear(in_features=1568, out_features=600)
    self.dropout= nn.Dropout(p=0.5)
    self.fc2 =nn.Linear(in_features=600, out_features=10)
  def forward(self,x):
    out =self.cnn1(x)
    out =self.batchnorm1(out)
    out =self.relu(out)
    out =self.maxpool1(out)
    out =self.cnn2(out)
    out =self.batchnorm2(out)
    out =self.relu(out)
    out =self.maxpool2(out)
    out =out.view(-1,1568)
    out =self.fc1(out)
    out =self.relu(out)
    out =self.dropout(out)
    out =self.fc2(out)
    return out

In [0]:
model=CNN()
CUDA=torch.cuda.is_available()
if CUDA:
  model=model.cuda()
loss_function=nn.CrossEntropyLoss()
optimizer=torch.optim.SGD(model.parameters(), lr=0.01)

In [0]:
iteration=0
for epoch in range(epochs):
  for i, (images,labels) in enumerate(train_load):
    iteration+=1
    if CUDA:
      images =Variable(images.cuda())
      labels =Variable(labels.cuda())
    else:
      images =Variable(images)
      labels =Variable(labels)
      
    optimizer.zero_grad()
    outputs=model(images)
    loss=loss_function(outputs,labels)
    loss.backward()
    optimizer.step()
    
    if(i+1)%100 ==0:
        correct =0
        total =0
        for images,labels in test_load:
            if CUDA:
              images =Variable(images.cuda())
            else:
              images =Variable(images)

            outputs=model(images)
            _,predicted=torch.max(outputs.data,1)
            total+=labels.size(0)
            if CUDA:
              correct += (predicted.cpu()==labels.cpu()).sum()
            else:
              correct += (predicted==labels).sum()

        accuracy = 100 *correct/total
        print("Iteration: {}, Train Loss: {}, Test Accuracy:{}%".format(iteration, loss.item(),accuracy))
        
print("Finished!")
    

Iteration: 100, Train Loss: 0.0857103243470192, Test Accuracy:98%
Iteration: 200, Train Loss: 0.20697177946567535, Test Accuracy:98%
Iteration: 300, Train Loss: 0.06593213230371475, Test Accuracy:98%
Iteration: 400, Train Loss: 0.0648646429181099, Test Accuracy:98%
Iteration: 500, Train Loss: 0.02223188430070877, Test Accuracy:98%
Iteration: 600, Train Loss: 0.032660093158483505, Test Accuracy:98%
Iteration: 700, Train Loss: 0.0625385269522667, Test Accuracy:98%
Iteration: 800, Train Loss: 0.09954184293746948, Test Accuracy:98%
Iteration: 900, Train Loss: 0.07626273483037949, Test Accuracy:98%
Iteration: 1000, Train Loss: 0.09606733918190002, Test Accuracy:98%
Iteration: 1100, Train Loss: 0.04067772999405861, Test Accuracy:98%
Iteration: 1200, Train Loss: 0.07098077982664108, Test Accuracy:98%
Iteration: 1300, Train Loss: 0.05420248582959175, Test Accuracy:98%
Iteration: 1400, Train Loss: 0.02750409208238125, Test Accuracy:98%
Iteration: 1500, Train Loss: 0.06596118211746216, Test Accu